# 线性回归的简洁实现

## 生成数据集

In [118]:
import numpy as np
import torch
from torch.utils import data
import d2l.torch as d2l

In [119]:
true_w = torch.tensor([2,-3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## 读取数据集

In [120]:
def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [121]:
batch_size = 10
data_iter = load_array((features,labels),batch_size)

In [122]:
next(iter(data_iter))

[tensor([[ 0.0113, -0.9103],
         [ 0.4683, -0.9330],
         [ 1.3758, -0.6145],
         [-1.5056, -0.3307],
         [-1.4590,  0.0919],
         [ 0.7053,  0.3318],
         [-1.4014, -0.3325],
         [ 1.0628, -0.6507],
         [-1.1701, -0.8366],
         [-0.8277,  1.1882]]),
 tensor([[ 7.3075],
         [ 8.3158],
         [ 9.0425],
         [ 2.3050],
         [ 0.9865],
         [ 4.4830],
         [ 2.5218],
         [ 8.5276],
         [ 4.7049],
         [-1.4921]])]

## 定义模型

In [123]:
from torch import nn

# Sequential将数据传入第一层，将第一层的输出作为第二层的输入
# 参数为多个模型层
# 这里只有一个输入形状为2，输出形状为1的全连接层
net = nn.Sequential(nn.Linear(2,1))

## 初始化参数

In [124]:
# net[0]表示网络中的第一层
# 使用 weight.data, bias.data 访问参数
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## 定义损失函数

In [125]:
# 均方损失函数  Mean Square Error Loss
loss = nn.MSELoss()
# 作业2:用HuberLoss()替代MSELoss()
#loss = nn.HuberLoss()

## 定义优化算法

In [126]:
trainer = torch.optim.SGD(net.parameters(),lr=0.03)

## 训练

In [127]:
num_epochs = 10
for epoch in range(num_epochs):
    for X,y in data_iter:
        # 对于每个batch，计算loss函数，并得到其梯度，参数减去单位学习率*梯度进行更新
        l = loss(net(X),y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    # 当所有batch遍历完后，该轮学习结束，计算该轮的损失函数
    l = loss(net(features),labels)
    # 在本轮的参数结果之上，进行下一轮学习
    print(f'epoch {epoch + 1}, loss {l:.10f}')

epoch 1, loss 2.2368545532
epoch 2, loss 0.4685491621
epoch 3, loss 0.0021268663
epoch 4, loss 0.0000584357
epoch 5, loss 0.0000521452
epoch 6, loss 0.0000523065
epoch 7, loss 0.0000522862
epoch 8, loss 0.0000523922
epoch 9, loss 0.0000523477
epoch 10, loss 0.0000522279


In [128]:
# 获取参数
w = net[0].weight.data
b = net[0].bias.data

# 求误差
print("w的误差：", true_w-w.reshape(true_w.shape)) #这里 true_w 为列向量，但 w 为行向量，无法直接操作
print("b的误差：", true_b-b)

w的误差： tensor([0.0001, 0.0001])
b的误差： tensor([0.0002])


In [129]:
# 获取参数梯度
net[0].weight.grad
net[0].bias.grad

tensor([0.0019])